# Question1: most important features for predicting rental prices

In [2]:
##In our classification model, the four most important features for predicting rental prices are as follows:


## 1. Bedrooms: The number of bedrooms is a significant determinant of rental prices, as it directly correlates with the capacity and comfort of the living space. 
# Properties with more bedrooms typically cater to larger households, making them more desirable and, consequently, commanding higher rental rates.


## 2. Postcode: The postcode serves as an indicator of the property's location, which significantly influences rental values. 
# Areas with desirable postcodes often offer better access to amenities, transportation, and schools, 
# contributing to higher demand and increased rental prices.


## 3. Bathrooms: The quantity of bathrooms is another crucial factor impacting rental pricing.
#  A higher number of bathrooms enhances convenience for tenants, 
# especially in larger households, and is often seen as a desirable feature that can justify a premium in rental costs.


## 4. Parking Spaces: The availability of parking spaces is a vital consideration in many regions, 
# particularly in urban settings where parking can be scarce. 
# Properties that offer designated parking tend to attract tenants who prioritize this convenience, 
# thereby increasing the potential rental value.